<a href="https://colab.research.google.com/github/tluxxx/weekly-pattern-in-stock-markets/blob/main/weekly_patterns_(part_3_walk_forward_testing_N_weeks_cycle_with_optimized_pattern).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exploiting weekly patterns in stock market for trading
#### part III: walk forward testing of N-weeks cycles in combination with M-years-training period) with detected and GA-optimized patterns within the cycles.

This series is based on a strategy-idea originally generated by Thomas Gebert (Kurzfrist-Strategien fuer Anleger, Boersenbuchverlag, 2020).
The original strategy was tested and expanded.  Further modifications are investigated. The modifications in this part are as follows:
1.   implementing walk forward test methods for 16 weeks cycle (detected pattern and GA optimized pattern) for PnL
1.   generalizing the aproach under 1 by looping trough various combinations of N and M and identifiying local PnL peaks
3.   applying the methodology under 1, but calculating the final equity value (i.e. considering a spesific start equity and flexible and fixed fees)
1.   generalizing the approach under 4 by looping through various combinations of N and M and identifying local peaks of the final equity values.



#1. Preparation and Data Upload

In [1]:
# loding external packages
!pip install pygad

# preparation
from google.colab import drive
drive.mount("/content/gdrive")
import sys
sys.path.append('/content/gdrive/MyDrive/Colab Notebooks/weekly_pattern')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.0 MB/s eta 0:00:00
Mounted at /content/gdrive


In [2]:
# importing all standard modules
import pandas as pd, numpy as np, yfinance as yf, numba as nb
import pygad, importlib
import plotly.express as px, plotly.graph_objects as go
from plotly.subplots import make_subplots
from tqdm import tqdm
from tabulate import tabulate
from warnings import filterwarnings
from numba import jit
filterwarnings('ignore')

In [4]:
# importing customized modules
from helpers_pattern_01 import *

In [ ]:
# re-importing customized modules (if required)
import helpers_pattern_01
importlib.reload(helpers_pattern_01)
from helpers_pattern_01 import *

In [5]:
# direct download price dataframe from yfinance
end_date = '2024-03-30'
start_date = '2000-01-01'
price = yf.download('^GDAXI', start=start_date, end=end_date)

[*********************100%%**********************]  1 of 1 completed


# 2. Walk Forward Optimisation - Part 1 (--> max PnL = w/o fees)

---



##2.1 Preparations

In [6]:
# conversion daily to weekly prices
price_w = transforming_daily_weekly(price, mode='open')

In [7]:
# helpers function
def walk_forward_1(price_w, m, n, limits):
  pos_x_year = []
  pos_test = pd.Series()
  price_w_test =pd.DataFrame()
  # walk forward
  for i, year_wf in enumerate(range(2000 + m, 2025)):
    # preparation training data( m years before current year_wf)
    years_train = list(range(year_wf - m, year_wf))
    price_w_train = price_w[price_w.index.year.isin(years_train)]
    # manual optimized pattern for train period (m years before current year_wf)
    df1, pos_x  = pattern_detection(price_w_train, n, upper_limit=limits[0], lower_limit=limits[1])
    # test data = current year (year_wf)
    price_w_test_x = price_w[price_w.index.year == year_wf]
    pos_w_test_x = pos_weekly(price_w_test_x, pos_x, shift=0)
    pos_test = pd.concat([pos_test, pos_w_test_x], axis=0)

  # final calculations for aggregated test period
  years_test = list(range(2000 + m, 2025))
  price_w_test = price_w[price_w.index.year.isin(years_test)]
  pnl_test = pnl_accumulation(price_w_test,pos_test)
  return pnl_test

## 2.2 PnL for 16-wks.-cycle and a given number of training years, walk forward calculationa and pattern detection (w/o fee)

In [8]:
# setting of parameter
n = 16                # cycle length
m = 8                # number of years in train cycles
limits = (0.2, -0.2)  # limits for pattern detection

In [9]:
# sample walk forward simulation
main_title = f'PnL of walk forward test for {n}-weeks cycle, test period {m}-years, detected pattern (w/o fee)'
sub_title = 'instrument: DAX [2000-2014]'
title = main_title + '<br><br><sup>' + sub_title + '</sup>'
# calculation
pnl_test_detect = walk_forward_1(price_w, m, n, limits)
years_test = list(range(2000 + m, 2025))
price_w_test = price_w[price_w.index.year.isin(years_test)]
pnl_buh = price_w_test['price'] / price_w_test['price'][0]

# plotting/printing
fig = go.Figure()
fig.add_trace(go.Scatter(x=pnl_test_detect.index, y=pnl_test_detect, name=f'walk forward, detected pattern {limits}'))
fig.add_trace(go.Scatter(x=pnl_buh.index, y=pnl_buh, name='Buy & Hold'))
fig.update_layout(template = 'plotly_dark', autosize=False, width=1200, height=600)
fig.update_layout(title=title, xaxis_title="Date", yaxis_title='PnL')
fig.show()

print(' ')
print('******************************************************')
print(f'Results detected pattern  {limits}:          {pnl_test_detect[-1]:.2f}')
print(f'Results Buy & Hold:                             {pnl_buh[-1]:.2f}')


 
******************************************************
Results detected pattern  (0.2, -0.2):          9.83
Results Buy & Hold:                             2.26


## 2.3 Final Pnl for combinations of {N = cycle length} and {M = length of training periods in years}, calculated using walk forward methods and pattern detection (w/o fees)

In [10]:
# testing of different combinations of weekly cycle length and length of training period.
n_range = range(5, 36)
m_range = range(2, 11)
n_x = list(n_range)
m_x = list(m_range)

result = np.zeros((len(n_x), len(m_x)))
for i, n in enumerate(tqdm(n_x)):
  for j, m in enumerate(m_x):
    pnl_test = walk_forward_1(price_w, m, n, limits)
    result[i][j] = (pnl_test[-1])

100%|██████████| 31/31 [00:27<00:00,  1.13it/s]


In [ ]:
# sample walk forward simulation
main_title = f'PnL of walk forward test for various N-weeks cycles and various lengths of the trainings period, detected pattern (w/o fee)'
sub_title = 'instrument: DAX [2000-2014]'
title = main_title + '<br><br><sup>' + sub_title + '</sup>'

# data for the surface plot
y, x = n_x, m_x
z = result

# surface plot
fig = go.Figure(data=[go.Surface(z=z, x=x, y=y)])
fig.update_layout(template = 'plotly_dark', autosize=False, width=1200, height=600)
fig.update_scenes(xaxis_title_text=' length of training period [y]',
                  yaxis_title_text=' length of cycles [wks]',
                  zaxis_title_text=' final PnL ')
fig.update_layout(title=title)
fig.show()

##2.4. PnL for 16 weeks cycle an a given length of the training periods in years, walking forward calculation; pattern within a training period optimized by GA

In [11]:
# function to be optimized via GA --> final pnl-values --> max
def opt_pattern1(ga_instance, pos_run, pos_run_idx):
  pnl_run = pnl_accumulation(pr_w_run,
                             pos_weekly(pr_w_run, pos_run, shift=0))
  return pnl_run[-1]

In [12]:
## Test all in ONe block
# setting of parameter
n = 16                # cycle length
m = 8                # number of years in train cycles
pos_x_year = []
pos_test = pd.Series()
price_w_test =pd.DataFrame()

# walk forward
for i, year_wf in enumerate(tqdm(range(2000 + m, 2025))):
  # preparation training data( m years before current year_wf)
  years_train = list(range(year_wf - m, year_wf))
  price_w_train = price_w[price_w.index.year.isin(years_train)]
  pr_w_run = price_w_train
  #print(pr_w_run)
  # manual optimized pattern for train period (m years before current year_wf)
  # definition of a new GA-instance
  ga_instance = pygad.GA(num_generations=100,
                         num_parents_mating=1,
                         fitness_func=opt_pattern1,
                         sol_per_pop=8,
                         num_genes=n,
                         gene_type=int,
                         gene_space=[-1, 0, 1],
                         mutation_percent_genes=10,
                         suppress_warnings=True)
  # running the GA across the training cycle data set and calculating the optimised pattern (=pos)
  ga_instance.run()
  pos_run, strat, strat_idx = ga_instance.best_solution()
  # test data = current year (year_wf)
  price_w_test_x = price_w[price_w.index.year == year_wf]
  pos_w_test_x = pos_weekly(price_w_test_x, pos_run, shift=0)
  pos_test = pd.concat([pos_test, pos_w_test_x], axis=0)

# final calculations for aggregated test period#
years_test = list(range(2000 + m, 2025))
price_w_test = price_w[price_w.index.year.isin(years_test)]
pnl_test_ga = pnl_accumulation(price_w_test,pos_test)

100%|██████████| 17/17 [00:30<00:00,  1.82s/it]


In [13]:
# sample walk forward simulation using GA
main_title = f'PnL of walk forward test for {n}-weeks cycle, test period {m}-years, GA optimized pattern (w/o fee)'
sub_title = 'instrument: DAX [2000-2014]'
title = main_title + '<br><br><sup>' + sub_title + '</sup>'

# calculation B&H for test_period
pnl_buh = price_w_test['price'] / price_w_test['price'][0]

# plotting/printing
fig = go.Figure()
fig.add_trace(go.Scatter(x=pnl_test_ga.index, y=pnl_test_ga, name=f'walk forward, GA-optimized pattern'))
fig.add_trace(go.Scatter(x=pnl_test_detect.index, y=pnl_test_detect, name=f'walk forward, detected pattern {limits}'))
fig.add_trace(go.Scatter(x=pnl_buh.index, y=pnl_buh, name='Buy & Hold'))
fig.update_layout(template = 'plotly_dark', autosize=False, width=1200, height=600)
fig.update_layout(title=title, xaxis_title="Date", yaxis_title='PnL')
fig.show()

print(' ')
print('******************************************************')

print(f'Results detected pattern                        {pnl_test_ga[-1]:.2f}')
print(f'Results detected pattern  {limits}:          {pnl_test_detect[-1]:.2f}')
print(f'Results Buy & Hold:                             {pnl_buh[-1]:.2f}')

 
******************************************************
Results detected pattern                        12.46
Results detected pattern  (0.2, -0.2):          9.83
Results Buy & Hold:                             2.26


## 2.5 Final Pnl for combinations of {N = cycle length} and {M = length of training periods in years}, calculated using walk forward methods, pattern within training period optimized using GA

In [14]:
# testing of different combinations of weekly cycle length and length of training period.
n_range = range(5, 36)
m_range = range(2, 11)
n_x = list(n_range)
m_x = list(m_range)

result = np.zeros((len(n_x), len(m_x)))
for k, n in enumerate(tqdm(n_x, leave=False)):
  for j, m in enumerate(tqdm(m_x, leave=False)):
    pos_x_year = []
    pos_test = pd.Series()
    price_w_test =pd.DataFrame()
    for i, year_wf in enumerate(range(2000 + m, 2025)):
      # preparation training data( m years before current year_wf)
      years_train = list(range(year_wf - m, year_wf))
      price_w_train = price_w[price_w.index.year.isin(years_train)]
      pr_w_run = price_w_train
      # manual optimized pattern for train period (m years before current year_wf)
      # definition of a new GA-instance
      ga_instance = pygad.GA(num_generations=70,
                             num_parents_mating=1,
                             fitness_func=opt_pattern1,
                             sol_per_pop=8,
                             num_genes=n,
                             gene_type=int,
                             gene_space=[-1, 0, 1],
                             mutation_percent_genes=10,
                             suppress_warnings=True)
      # running the GA across the training cycle data set and calculating the optimised pattern (=pos)
      ga_instance.run()
      pos_run, strat, strat_idx = ga_instance.best_solution()
      # test data = current year (year_wf)
      price_w_test_x = price_w[price_w.index.year == year_wf]
      pos_w_test_x = pos_weekly(price_w_test_x, pos_run, shift=0)
      pos_test = pd.concat([pos_test, pos_w_test_x], axis=0)

    # final calculations for aggregated test period#
    years_test = list(range(2000 + m, 2025))
    price_w_test = price_w[price_w.index.year.isin(years_test)]
    pnl_test_ga = pnl_accumulation(price_w_test,pos_test)
    result[k][j] = (pnl_test_ga[-1])

 11%|█         | 1/9 [00:24<03:13, 24.20s/it]


KeyboardInterrupt: 

In [ ]:
# sample walk forward simulation
main_title = f'PnL of walk forward test for various N-weeks cycles and various lenthgs of the training period, GA-optimized pattern (w/o fee)'
sub_title = 'instrument: DAX [2000-2024]'
title = main_title + '<br><br><sup>' + sub_title + '</sup>'

# data for the surface plot
y, x = n_x, m_x
z = result

# surface plot
fig = go.Figure(data=[go.Surface(z=z, x=x, y=y)])
fig.update_layout(template = 'plotly_dark', autosize=False, width=1200, height=600)
fig.update_scenes(xaxis_title_text=' length of training period [y]',
                  yaxis_title_text=' length of cycles [wks]',
                  zaxis_title_text=' final PnL ')
fig.update_layout(title=title)
fig.show()

# 3. Walk-Forward-Optimization Part 2 ( --> max final equity = w fee)


*   trading-times: Mondays at open (9:00)
*   GA via PyGAD-module
*   maximizing final equity (w fee)




## 3.1 Setting of parameters and helper functions

In [18]:
# main parameter settings and preparations:
# assumtion concerning equity and fee
start_equity = 10000                  # equity at start
fee = 0.0025                          # 0.25% of equity per trade
fixed_fee = 4.9                       # 4.9 EUR per trade

# conversion daily to weekly prices
price_w = transforming_daily_weekly(price, mode='open')

In [19]:
# function to be optimized via GA --> final equity --> max
def opt_pattern2(ga_instance, pos_run, pos_run_idx):
  eq_run = pnl_acc_real_equity(pr_w_run,
                               pos_weekly(pr_w_run, pos_run, shift=0),
                               start_equity, fee, fixed_fee)
  return eq_run[-1]

In [20]:
# helpers function
def walk_forward_2(price_w, m, n, limits):
  pos_x_year = []
  pos_test = pd.Series()
  price_w_test =pd.DataFrame()
  # walk forward
  for i, year_wf in enumerate(range(2000 + m, 2025)):
    # preparation training data( m years before current year_wf)
    years_train = list(range(year_wf - m, year_wf))
    price_w_train = price_w[price_w.index.year.isin(years_train)]
    # manual optimized pattern for train period (m years before current year_wf)
    df1, pos_x  = pattern_detection(price_w_train, n, upper_limit=limits[0], lower_limit=limits[1])
    # test data = current year (year_wf)
    price_w_test_x = price_w[price_w.index.year == year_wf]
    pos_w_test_x = pos_weekly(price_w_test_x, pos_x, shift=0)
    pos_test = pd.concat([pos_test, pos_w_test_x], axis=0)

  # final calculations for aggregated test period
  years_test = list(range(2000 + m, 2025))
  price_w_test = price_w[price_w.index.year.isin(years_test)]
  pnl_test = pnl_accumulation(price_w_test,pos_test)
  equity_x = pnl_acc_real_equity(price_w_test, pos_test, start_equity, fee, fixed_fee)
  return equity_x

##3.2 Calculation of final Equity using the already detected patterns, w fee

### 3.2.1 Final Equity for single pair of {N = cycle length} and {M = length of training periods in years}, calculated using walk forward method, pattern = detect, w fee

In [21]:
# setting of parameter (equity calculation for a given N, M combination )
n = 16                # cycle length
m = 8                # number of years in train cycles
limits = (0.2, -0.2)  # limits for pattern detection

In [29]:
# sample walk forward simulation
main_title = f'final Equity, walk forward test for {n}-weeks cycle & {m}-years-train-period, pattern = detected, w fee'
sub_title = 'instrument: DAX [2000-2014]'
title = main_title + '<br><br><sup>' + sub_title + '</sup>'
# calculation
years_test = list(range(2000 + m, 2025))
price_w_test = price_w[price_w.index.year.isin(years_test)]
x = walk_forward_2(price_w, m, n, limits)
equity_test_detect = pd.Series(x, index=price_w_test.index)
equity_buh = price_w_test['price'] / price_w_test['price'][0] * start_equity

# plotting/printing
fig = go.Figure()
fig.add_trace(go.Scatter(x=equity_test_detect.index, y=equity_test_detect, name=f'walk forward, detected pattern {limits}'))
fig.add_trace(go.Scatter(x=equity_buh.index, y=equity_buh, name='Buy & Hold'))
fig.update_layout(template = 'plotly_dark', autosize=False, width=1200, height=600)
fig.update_layout(title=title, xaxis_title="Date", yaxis_title='PnL')
fig.show()

print(' ')
print('******************************************************')
print(f'Results detected pattern  {limits}:          {equity_test_detect[-1]:.2f}')
print(f'Results Buy & Hold:                             {equity_buh[-1]:.2f}')

 
******************************************************
Results detected pattern  (0.2, -0.2):          9947.28
Results Buy & Hold:                             22627.92


### 3.2.2 Final Equity for various combinations of {N = cycle length} and {M = length of training periods in years}, calculated using walk forward method, pattern = detect, w fee

In [30]:
# testing of different combinations of weekly cycle length and length of training period.
n_range = range(5, 36)
m_range = range(2, 11)
n_x = list(n_range)
m_x = list(m_range)

result = np.zeros((len(n_x), len(m_x)))
for i, n in enumerate(tqdm(n_x)):
  for j, m in enumerate(m_x):
    equity_test = walk_forward_2(price_w, m, n, limits)
    result[i][j] = (equity_test[-1])

100%|██████████| 31/31 [00:26<00:00,  1.17it/s]


In [32]:
# sample walk forward simulation
main_title = f'final Equity, walk forward backtest for various N-weeks cycles and various lengths of the trainings period, detected pattern, with fee'
sub_title = 'instrument: DAX [2000-2014]'
title = main_title + '<br><br><sup>' + sub_title + '</sup>'

# data for the surface plot
y, x = n_x, m_x
z = result

# surface plot
fig = go.Figure(data=[go.Surface(z=z, x=x, y=y)])
fig.update_layout(template = 'plotly_dark', autosize=False, width=1200, height=600)
fig.update_scenes(xaxis_title_text=' length of training period [y]',
                  yaxis_title_text=' length of cycles [wks]',
                  zaxis_title_text=' final Equity ')
fig.update_layout(title=title)
fig.show()

##3.3 Final Equity for combinations of {N = cycle length} and {M = length of training periods in years}, calculated using walk forward method, pattern: GA optimized, w fee

In [45]:
# testing of different combinations of weekly cycle length and length of training period.

result_ga = np.zeros((len(n_x), len(m_x)))
result_ga_pos = np.zeros((len(n_x), len(m_x)), dtype=np.ndarray)

for k, n in enumerate(tqdm(n_x, leave=False)):
  for j, m in enumerate(tqdm(m_x, leave=False)):
    pos_x_year = []
    pos_test = pd.Series()
    price_w_test =pd.DataFrame()
    for i, year_wf in enumerate(range(2000 + m, 2025)):
      # preparation training data( m years before current year_wf)
      years_train = list(range(year_wf - m, year_wf))
      price_w_train = price_w[price_w.index.year.isin(years_train)]
      pr_w_run = price_w_train
      # manual optimized pattern for train period (m years before current year_wf)
      # definition of a new GA-instance
      ga_instance = pygad.GA(num_generations=70,
                             num_parents_mating=1,
                             fitness_func=opt_pattern2,
                             sol_per_pop=8,
                             num_genes=n,
                             gene_type=int,
                             gene_space=[-1, 0, 1],
                             mutation_percent_genes=10,
                             suppress_warnings=True)
      # running the GA across the training cycle data set and calculating the optimised pattern (=pos)
      ga_instance.run()
      pos_run, strat, strat_idx = ga_instance.best_solution()
      # test data = current year (year_wf)
      price_w_test_x = price_w[price_w.index.year == year_wf]
      pos_w_test_x = pos_weekly(price_w_test_x, pos_run, shift=0)
      pos_test = pd.concat([pos_test, pos_w_test_x], axis=0)

    # final calculations for aggregated test period
    years_test = list(range(2000 + m, 2025))
    price_w_test = price_w[price_w.index.year.isin(years_test)]
    equity_ga = pnl_acc_real_equity(price_w_test, pos_test, start_equity, fee, fixed_fee)
    result_ga[k][j] = (equity_ga[-1])
    result_ga_pos[k][j] = pos_test

100%|██████████| 3/3 [01:36<00:00, 31.64s/it]


In [46]:
# sample walk forward simulation
main_title = 'final equity, walk forward test for selected combination of {N = cycle length} and {M = length of train}, pattern = GA-optimized, w fee'
sub_title = 'instrument: DAX [2000-2024]'
title = main_title + '<br><br><sup>' + sub_title + '</sup>'

# data for the surface plot
y, x = n_x, m_x
z = result_ga_w_fee

# surface plot
fig = go.Figure(data=[go.Surface(z=z, x=x, y=y)])
fig.update_layout(template = 'plotly_dark', autosize=False, width=1200, height=600)
fig.update_scenes(xaxis_title_text=' length of training period [y]',
                  yaxis_title_text=' length of cycles [wks]',
                  zaxis_title_text=' final equity ')
fig.update_layout(title=title)
fig.show()

In [64]:
# selection of value out of the lists n_x= [8, 12, 16, 22, 32] an m_x = [6, 8, 10]
k = 2
j = 1
n = n_x[k]
m = m_x[j]
# sample walk forward simulation
main_title = f'final Equity, walk forward test for {n}-weeks cycle & {m}-years-train-period, pattern = GA-optimized, w fee'
sub_title = 'instrument: DAX [2000-2024]'
title = main_title + '<br><br><sup>' + sub_title + '</sup>'
# calculation
years_test = list(range(2000 + m, 2025))
price_w_test = price_w[price_w.index.year.isin(years_test)]
pos_x = pd.Series(result_ga_pos[k][j].tolist() ,
                  index=price_w_test.index)
equity_test_ga = pd.Series(pnl_acc_real_equity(price_w_test, pos_x, start_equity, fee, fixed_fee),
                           index=price_w_test.index)
equity_buh = price_w_test['price'] / price_w_test['price'][0] * start_equity

# plotting/printing
fig = go.Figure()
fig.add_trace(go.Scatter(x=equity_test_ga.index, y=equity_test_ga, name=f'walk forward, pattern: GA-optimized'))
fig.add_trace(go.Scatter(x=equity_buh.index, y=equity_buh, name='Buy & Hold'))
fig.update_layout(template = 'plotly_dark', autosize=False, width=1200, height=600)
fig.update_layout(title=title, xaxis_title="Date", yaxis_title='PnL')
fig.show()

print(' ')
print('******************************************************')
print(f'Results detected pattern :                  {equity_test_ga[-1]:.2f}')
print(f'Results Buy & Hold:                         {equity_buh[-1]:.2f}')

 
******************************************************
Results detected pattern :                  21557.63
Results Buy & Hold:                         22627.92


In [63]:
#n_x =
#m_x =
# all combinations fro n_x = [8, 12, 16, 22, 32] and m_x = [6, 8, 10]
main_title = 'final Equity, walk forward test for {N=length weeks cycle & {m = years-train-period}, pattern = GA-optimized, w fee'
sub_title = 'instrument: DAX [2000-2024]'
title = main_title + '<br><br><sup>' + sub_title + '</sup>'

# plotting/printing
fig = go.Figure()
print('***********************************************')

# calculation
for k, n  in enumerate(n_x):
  for j, m in enumerate(m_x):
    # calculation
    years_test = list(range(2000 + m, 2025))
    price_w_test = price_w[price_w.index.year.isin(years_test)]
    pos_x = pd.Series(result_ga_pos[k][j].tolist() ,
                      index=price_w_test.index)
    equity_test_ga = pd.Series(pnl_acc_real_equity(price_w_test, pos_x, start_equity, fee, fixed_fee),
                               index=price_w_test.index)
    equity_test_buh = price_w_test['price'] / price_w_test['price'][0] * start_equity
    fig.add_trace(go.Scatter(x=equity_test_ga.index, y=equity_test_ga, name=f'{n}-weeks-cycle, {m}-years train period'))
    print(f'final equity, pattern: GA optimized ({n}-weeks-cycle, {m}-years train period): {equity_test_ga[-1]:.2f}   Final Equity Buy & Hold (relevant period [{years_test[0]} - 2024]: {equity_test_buh[-1]:,.2f}')

fig.update_layout(template = 'plotly_dark', autosize=False, width=1200, height=600)
fig.update_layout(title=title, xaxis_title="Date", yaxis_title='PnL')

print(' ')
fig.show()



***********************************************
final equity, pattern: GA optimized (8-weeks-cycle, 6-years train period): 8261.70   Final Equity Buy & Hold (relevant period [2006 - 2024]: 33,651.67
final equity, pattern: GA optimized (8-weeks-cycle, 8-years train period): 8152.61   Final Equity Buy & Hold (relevant period [2008 - 2024]: 22,627.92
final equity, pattern: GA optimized (8-weeks-cycle, 10-years train period): 18359.94   Final Equity Buy & Hold (relevant period [2010 - 2024]: 30,468.24
final equity, pattern: GA optimized (12-weeks-cycle, 6-years train period): 1346.53   Final Equity Buy & Hold (relevant period [2006 - 2024]: 33,651.67
final equity, pattern: GA optimized (12-weeks-cycle, 8-years train period): 3293.15   Final Equity Buy & Hold (relevant period [2008 - 2024]: 22,627.92
final equity, pattern: GA optimized (12-weeks-cycle, 10-years train period): 1934.02   Final Equity Buy & Hold (relevant period [2010 - 2024]: 30,468.24
final equity, pattern: GA optimized (16-